In [2]:
import pandas
import time
import github
import gpt

In [3]:
# reading the CSV file
csvFile = pandas.read_csv('dataset/dataset_sample.csv')

# Pre-processing the CSV file
csvFile = csvFile[csvFile['GitHub_Repo_Link'].notna()]

csvFile

,Index,GitHub_Repo_Link,GitHub_Build_Pipeline_File_Content,Generated_Build_Pipeline_File_Content,Exact_Match_Score,BLEU_Score,Syntax_Check,DevOps_Aware_Score
0,1.0,https://github.com/DapperLib/Dapper,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,https://github.com/huiyadanli/RevokeMsgPatcher,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,https://github.com/jasontaylordev/CleanArchite...,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,https://github.com/nilaoda/N_m3u8DL-CLI,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,https://github.com/shadowsocks/shadowsocks-win...,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def run_experiment(csvFile):
    for i in range(0,5):
        repo_identifier = csvFile.iloc[i]['GitHub_Repo_Link'].split('github.com/')[1]
        print(repo_identifier)
        try:
            default_branch = github.get_default_branch(repo_identifier);
            repo_structure = github.get_repository_tree(repo_identifier, default_branch)
            dependencies = github.get_list_of_dependencies(repo_identifier)
            csvFile.loc[i,'Generated_Build_Pipeline_File_Content'] = gpt.generate_build_pipeline(repo_structure, dependencies, default_branch)
            # Add delay to avoid rate limiting
            time.sleep(10)
        except Exception as e:
            print(e)
            continue

    csvFile.to_csv('DevOps_LLM_Bot_Test_Data - C#.csv', index=False)

In [13]:
run_experiment(csvFile)

DapperLib/Dapper
huiyadanli/RevokeMsgPatcher
jasontaylordev/CleanArchitecture
nilaoda/N_m3u8DL-CLI
shadowsocks/shadowsocks-windows
